In [1]:
# Setup
!pip install alpaca-trade-api --break-system-packages && pip install alpaca-py --break-system-packages # Alpaca Trading Data

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
# Config
import sys
sys.path.append('./config')
import config

In [10]:
# Parameters
timeframe = "1d"
pairs = [
     "BTC/USDT",    # "BTC/USDT:USDT",# "ETH/USDT:USDT",BTC/USDT,ETH/USDT:USDT,ADA/USDT:USDT,AVAX/USDT:USDT,SOL/USDT:USDT,LINK/USDT:USDTmBTC/USDT,BTC/USDT,/USDT:USDT
]
start_date = "2023-01-01" # 00:00:00"
end_date = "2025-08-13" # 00:00:00"

In [11]:
# Data Download and Save
from utilities.data_manager import DataManager
data = DataManager(config.EXCHANGE_NAME, config.BASE_DATA_PATH)

for pair in pairs:
    data.download(pair, timeframe, start_date, end_date)

# It's Overwriting (not merging- ok maybe it is?), it goes into an individual timeframe/ticker.csv file, issue with timeframe handling date/time

fetched binance ohlcv data for BTC/USDT from 2025-08-16 08:00:00
Saved 958 new rows to BTC-USDT.csv


In [12]:
# Data Load
for pair in pairs:
    ohlcv = data.load(pair, timeframe, start_date, end_date) #"2025-01-01 00:00:00", end_date="2025-02-18 00:00:00")
    #print(f"{pair} \n {ohlcv} \n")

In [13]:
# Data Preprocessing - Technical Analysis

In [18]:
# Strategy
import importlib # Required to Import Code Library from Folder
from scenarios.scenarios_loader import load_scenario # Import the Scenario Loader
scenario = load_scenario("btc_macd_daily_full") #btc_sma_daily btc_macd_daily_full
module_path = f"strategies.{scenario.strategy_name}" # Strategy path ./strategies/strategy_name
strat = importlib.import_module(module_path) # Import code dynamically from the strategies folder

# Load OHLCV Data for the Scenario
ohlcv = data.load(pairs[0], timeframe=timeframe, start_date=start_date)

consideration = strat.Strategy(scenario.strategy_params, ohlcv)
# Assume strategy has already been instantiated
last_row = consideration.data.iloc[-1]

if last_row["long_entry"]:
    print("📈 Signal: BUY (Golden Cross)")

elif last_row["short_entry"]:
    print("📉 Signal: SELL (Death Cross)")

else:
    print("⏸️ No signal")

⏸️ No signal


In [16]:
# Execution

In [8]:
# Refine Strategy
# Load the Strategy Scenario TODO: Move out dates and ticker from strategy scenatio to parameters, TODO2: Remove Technical Analyssi for Strategy to Data Preprocessing
import importlib # Required to Import Code Library from Folder
from scenarios.scenarios_loader import load_scenario # Import the Scenario Loader
scenario = load_scenario("btc_macd_daily_full") #btc_sma_daily btc_macd_daily_full
module_path = f"strategies.{scenario.strategy_name}" # Strategy path ./strategies/strategy_name
strat = importlib.import_module(module_path) # Import code dynamically from the strategies folder

# Load OHLCV Data for the Scenario
#ohlcv = data.load(scenario.symbol, timeframe=scenario.timeframe, start_date=scenario.start_date)
ohlcv = data.load(pairs[0], timeframe=timeframe, start_date=start_date)

# Backtest the Strategy Scenario
backtest = strat.Strategy(scenario.strategy_params, ohlcv)
backtest.run_backtest(
    initial_balance=scenario.initial_balance,
    leverage=scenario.leverage,
    open_fee_rate=scenario.open_fee_rate,
    close_fee_rate=scenario.close_fee_rate
)

from utilities.backtest_analysis import BacktestAnalysis # TODO: Backtest needs to compare to HODL
results = BacktestAnalysis(backtest) # Analyse the Results from the Backtest for the Scenario Strategy
#results.print_metrics() # Prints Results Summary to Screen
results.save_metrics_as_csv(scenario) # Appends to Results.csv

from IPython.display import display # Force Jupyter to render the DataFrame in its rich HTML table format
display(backtest.trades_info) # Print/Display the Trades, requires it's own block
display(backtest.equity_record) # Print/Display the Equity Record, requires it's own block

# Apply Recommended Scenario
recommended_scenario = backtest

# Hypothesize Scenarios

/home/spoonbill/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


,open_time,close_time,open_reason,close_reason,open_price,close_price,initial_margin,net_pnl,net_pnl_pct,open_notional_value,close_notional_value,amount,open_fee,close_fee,sl_price,liquidation_price,open_balance,close_balance
0,2023-10-04,2023-10-14,Open long,MACD Death Cross,27778.57,26852.48,1000.0,-34.111503,-3.411150,999.8,966.468378,0.035992,0.2,0.579881,19444.999,0.0,0.000000,965.888497
1,2023-10-16,2024-01-15,Open long,MACD Death Cross,28500.78,42511.10,1000.0,490.383639,49.038364,999.8,1491.278406,0.035080,0.2,0.894767,19950.546,0.0,-34.111503,1456.272137
2,2024-02-10,2024-04-13,Open long,MACD Death Cross,47751.09,63924.51,1000.0,337.631833,33.763183,999.8,1338.434894,0.020938,0.2,0.803061,33425.763,0.0,456.272137,1793.903970
3,2024-06-06,2024-06-07,Open long,MACD Death Cross,70799.06,69355.60,1000.0,-21.171696,-2.117170,999.8,979.415954,0.014122,0.2,0.587650,49559.342,0.0,793.903970,1772.732274
4,2024-07-23,2024-08-02,Open long,MACD Death Cross,65936.01,61498.33,1000.0,-68.048875,-6.804887,999.8,932.510632,0.015163,0.2,0.559506,46155.207,0.0,772.732274,1704.683400
5,2024-09-20,2024-12-28,Open long,MACD Death Cross,63201.05,95300.00,1000.0,506.680212,50.668021,999.8,1507.584763,0.015819,0.2,0.904551,44240.735,0.0,704.683400,2211.363612
6,2025-01-21,2025-01-31,Open long,MACD Death Cross,106143.82,102429.56,1000.0,-35.764601,-3.576460,999.8,964.814288,0.009419,0.2,0.578889,74300.674,0.0,1211.363612,2175.599011
7,2025-04-22,2025-06-20,Open long,MACD Death Cross,93442.99,103297.99,1000.0,104.581136,10.458114,999.8,1105.244282,0.010700,0.2,0.663147,65410.093,0.0,1175.599011,2280.180147
8,2025-07-03,2025-08-03,Open long,MACD Death Cross,109584.78,114208.80,1000.0,41.362192,4.136219,999.8,1041.987384,0.009124,0.2,0.625192,76709.346,0.0,1280.180147,2321.542338


,price,equity
time,,
2023-01-01,16616.75,1000.000000
2023-01-02,16672.87,1000.000000
2023-01-03,16675.18,1000.000000
2023-01-04,16850.36,1000.000000
2023-01-05,16831.85,1000.000000
...,...,...
2025-08-11,118686.00,2321.542338
2025-08-12,120134.08,2320.742458
2025-08-13,123306.43,2347.128081


In [ ]:
# Reporting